<h1><strong>Análise de sentimentos em avaliações de e-commerce de roupas</strong></h1>


**Objetivo**:

Classificar as avaliações de e-commerce de roupas em positiva ou negativa.

*   Dataset: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

***Orientador:*** Thiago de Melo

***Orientando:*** Edson Brilhante

# Importando as bibliotecas

In [1]:
!pip install auto-sklearn

In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import autosklearn.classification
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Importando o *dataset*

In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Edson Henrique/dataset/Womens Clothing E-Commerce Reviews.csv')

# Visualização do *dataset*

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


# Modelagem

## Seleciona as colunas '*Review Text*' e '*Rating*'

In [5]:
dataset = dataset[['Review Text', 'Rating']]
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22641 entries, 0 to 22640
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  22641 non-null  object
 1   Rating       22641 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 353.9+ KB


## Encoding

Seleciona o atributo *'Rating'* e o converte da seguinte forma:



*   1, 2 e 3: negativa
*   4 e 5: positiva

In [6]:
dataset['Rating'] = dataset['Rating'].apply(lambda x: 1 if x >= 4 else 0)
#dataset['Rating'] = pd.Categorical(dataset['Rating'])

In [7]:
dataset

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
22636,I was very happy to snag this dress at such a ...,1
22637,"It reminds me of maternity clothes. soft, stre...",0
22638,"This fit well, but the top was very see throug...",0
22639,I bought this dress for a wedding i have this ...,0


## Limpando as avaliações

In [8]:
english_stopwords = list()
with open('/content/drive/MyDrive/Edson Henrique/code/stopwords/english', 'r') as a_file:
  for line in a_file:
    stripped_line = line.strip()
    english_stopwords.append(stripped_line)

excluded_words = ("not", "no",  'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'shouldn', "shouldn't",  "won't", 'wouldn', "wouldn't")
english_stopwords = [x for x in english_stopwords if x not in excluded_words]
print(english_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very',

In [9]:
corpus = []
for i in range (0, dataset.shape[0]):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review Text'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(english_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [10]:
corpus[0]

'absolut wonder silki sexi comfort'

In [11]:
dataset.iloc[0]

Review Text    Absolutely wonderful - silky and sexy and comf...
Rating                                                         1
Name: 0, dtype: object

## Criando o modelo *bag of words*

In [12]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

## Dividindo o *dataset* em treino e teste

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Treino

In [14]:
cls = autosklearn.classification.AutoSklearnClassifier()
cls.fit(X_train, y_train)

[WARNING] [2022-02-16 15:17:48,870:Client-AutoML(1):999a1822-8f3b-11ec-815b-0242ac1c0002] Unknown dtype for X: int64, assuming it takes 8 bit/number


AutoSklearnClassifier(per_run_time_limit=360)

## Teste

In [26]:
y_pred = cls.predict(X_test)

In [25]:
print('Matriz de confusão')
print(confusion_matrix(y_test, y_pred))

print('\nMétricas')
print(classification_report(y_test, y_pred, digits=3))

print('\nScore')
print(accuracy_score(y_test, y_pred))

Matriz de confusão
[[ 668  449]
 [ 184 3228]]

Métricas
              precision    recall  f1-score   support

           0      0.784     0.598     0.679      1117
           1      0.878     0.946     0.911      3412

    accuracy                          0.860      4529
   macro avg      0.831     0.772     0.795      4529
weighted avg      0.855     0.860     0.853      4529


Score
0.8602340472510488
